In [1]:
import pandas as pd
import numpy as np

In [2]:
proj_list= pd.read_csv('Example3_ProjList.csv', index_col=['ID']).fillna(0)
proj_list

,Programme,Type,Location,Region,CAPEX,NPV,CAPEX Yr1,CAPEX Yr2,CAPEX Yr3,CAPEX Yr4,Relationship,RelationshipProjID
ID,,,,,,,,,,,,
1,Piping,AssetIntegrity,Facility J,Zone4,5,6,4,1,0,0,Contingent,2.0
2,Instrumentation,AssetIntegrity,Facility F,Zone1,10,11,1,5,4,0,Contingent,1.0
3,Mechanical,AssetIntegrity,Facility C,Zone3,10,14,2,4,4,0,Mutually_Exclusive,5.0
4,Debottlenecking,Growth,Facility D,Zone4,5,7,5,0,0,0,0,0.0
5,Piping,AssetIntegrity,Facility C,Zone3,10,12,4,4,2,0,Mutually_Exclusive,3.0
6,Expansion,Growth,Facility C,Zone3,15,20,7,8,0,0,0,0.0
7,Revamp,Growth,Facility F,Zone1,15,16,7,7,1,0,0,0.0
8,Piping,AssetIntegrity,Facility D,Zone4,2,5,1,1,0,0,Mandatory,8.0
9,Revamp,Growth,Facility E,Zone5,3,4,2,1,0,0,Mandatory,9.0


In [3]:
capexconstraints=pd.read_csv('Example3_AnnualCAPEX.csv')['CAPEX'].values.tolist()
capexconstraints

[50, 70, 40, 20]

In [4]:
#Sensitivity Run
capexconstraints_10pctoverrunallowance = [round(i * 1.1,1) for i in capexconstraints]

#Uncomment below to conduct a sensitivity of 'relaxing' the CAPEX threshold by allowing 10% Overruns 

capexconstraints=capexconstraints_10pctoverrunallowance 

In [5]:
relationships=proj_list[['Relationship','RelationshipProjID']].dropna(thresh=2)
relationships['RelationshipProjID2']=relationships.index
MutuallyExclusive=relationships.loc[relationships['Relationship'] == 'Mutually_Exclusive'].sort_values(['RelationshipProjID2'])
MutuallyExclProjectsList=list(set(map(lambda x: tuple(sorted(x)),[[i , j] for i, j in zip(MutuallyExclusive['RelationshipProjID2'].values.tolist(), MutuallyExclusive['RelationshipProjID'].values.tolist())])))
Contingent=relationships.loc[relationships['Relationship'] == 'Contingent'].sort_values(['RelationshipProjID2'])
ContingentProjectsList = list(set(map(lambda x: tuple(sorted(x)),[[i , j] for i, j in zip(Contingent['RelationshipProjID2'].values.tolist(), Contingent['RelationshipProjID'].values.tolist())])))
Mandatory=relationships.loc[relationships['Relationship'] == 'Mandatory'].sort_values(['RelationshipProjID2'])
MandatoryProjectsList = Mandatory['RelationshipProjID2'].values.tolist()

In [6]:
MutuallyExclProjectsList

[(19, 20.0), (3, 5.0)]

In [7]:
ContingentProjectsList

[(1, 2.0), (26, 29.0)]

In [8]:
MandatoryProjectsList 

[8, 9, 14, 16]

In [9]:
import pulp

# Create A Model

phasing = pulp.LpProblem("Maximise", pulp.LpMaximize)

Selection = pulp.LpVariable.dicts("Selection", proj_list.index, cat='Binary')

In [10]:
# Set The Objective Function                                                                                                         
phasing += pulp.lpSum(Selection[idx]*proj_list.loc[idx]["NPV"] for idx in proj_list.index)

In [11]:
# Loop over for mutually exclusive projects
for i in range(0,len(MutuallyExclProjectsList)):
    phasing += Selection[MutuallyExclProjectsList[i][0]] + Selection[MutuallyExclProjectsList[i][1]] <= 1
                                                                     
# Loop over contingent projects
for j in range(0,len(ContingentProjectsList)):
    phasing += Selection[ContingentProjectsList[j][0]] == Selection[ContingentProjectsList[j][1]] 

# Loop over mandatory projects

for k in range(0,len(MandatoryProjectsList)):
    phasing += Selection[MandatoryProjectsList[k]] == 1

In [12]:
#Set The Constraints 

for m in range(0,len(capexconstraints)):
    year=str(m+1)
    phasing += sum([Selection[idx] * proj_list.loc[idx]["CAPEX Yr"+year] for idx in proj_list.index]) <= capexconstraints[m]


In [13]:
# Run The Solver(s)

%time phasing.solve() #equivalent to phasing.solve(pulp.PULP_CBC_CMD()) as CBC is PulP's default solver

pulp.LpStatus[phasing.status]

# Print our objective function value and Output Solution
print (pulp.value(phasing.objective))


Wall time: 312 ms
268.0


In [14]:
#Convert output into user friendly output for viewing or downloading 
outputtablecolumns=[]
outputtablecolumns.append('Selection Y/N')
for n in range(0,len(capexconstraints)):
    year=str(n+1)
    outputtablecolumns.append("CAPEX Yr"+year+" Selected")
outputtablecolumns.append('NPV Selected')


pulpsolution=pd.DataFrame()
pulpsolution['Selection Y/N']= [Selection[idx].value() for idx in proj_list.index]

for n in range(0,len(capexconstraints)):
    year=str(n+1)
    pulpsolution["CAPEX Yr"+year+" Selected"]= [Selection[idx].value()*proj_list.loc[idx]["CAPEX Yr"+year] for idx in proj_list.index]

pulpsolution['NPV Selected']= [Selection[idx].value()*proj_list.loc[idx]["NPV"] for idx in proj_list.index]

pulpsolution

pulpsolution.index += 1 
pulpsolution

pulpoutput = pd.concat([proj_list, pulpsolution], axis=1)
pulpoutput.drop(['CAPEX Yr1', 'CAPEX Yr2','CAPEX Yr3'], axis=1)
    

,Programme,Type,Location,Region,CAPEX,NPV,CAPEX Yr4,Relationship,RelationshipProjID,Selection Y/N,CAPEX Yr1 Selected,CAPEX Yr2 Selected,CAPEX Yr3 Selected,CAPEX Yr4 Selected,NPV Selected
1,Piping,AssetIntegrity,Facility J,Zone4,5,6,0,Contingent,2.0,1.0,4.0,1.0,0.0,0.0,6.0
2,Instrumentation,AssetIntegrity,Facility F,Zone1,10,11,0,Contingent,1.0,1.0,1.0,5.0,4.0,0.0,11.0
3,Mechanical,AssetIntegrity,Facility C,Zone3,10,14,0,Mutually_Exclusive,5.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Debottlenecking,Growth,Facility D,Zone4,5,7,0,0,0.0,1.0,5.0,0.0,0.0,0.0,7.0
5,Piping,AssetIntegrity,Facility C,Zone3,10,12,0,Mutually_Exclusive,3.0,1.0,4.0,4.0,2.0,0.0,12.0
6,Expansion,Growth,Facility C,Zone3,15,20,0,0,0.0,1.0,7.0,8.0,0.0,0.0,20.0
7,Revamp,Growth,Facility F,Zone1,15,16,0,0,0.0,1.0,7.0,7.0,1.0,0.0,16.0
8,Piping,AssetIntegrity,Facility D,Zone4,2,5,0,Mandatory,8.0,1.0,1.0,1.0,0.0,0.0,5.0
9,Revamp,Growth,Facility E,Zone5,3,4,0,Mandatory,9.0,1.0,2.0,1.0,0.0,0.0,4.0
10,Piping,AssetIntegrity,Facility H,Zone3,7,9,0,0,0.0,1.0,3.0,3.0,1.0,0.0,9.0


In [15]:
yearSumCapexColumns=[]
for p in range(0,len(capexconstraints)):
    year=str(p+1)
    yearSumCapexColumns.append("CAPEX Yr"+year+" Selected")
yearSumCapexColumns

CAPEX_Totals=[pulpsolution[yr].sum() for yr in yearSumCapexColumns]

CAPEX_CheckSum= pd.DataFrame()
CAPEX_CheckSum['Year'] = np.arange(len(yearSumCapexColumns))+1
CAPEX_CheckSum['CAPEX Phasing'] = CAPEX_Totals
CAPEX_CheckSum['Constraints'] = capexconstraints
CAPEX_CheckSum['Budget Utilisation%'] = round(CAPEX_CheckSum['CAPEX Phasing']/CAPEX_CheckSum['Constraints'],2)*100

print("Summary")
print()
print('Total NPV=', round(pulpsolution['NPV Selected'].sum(),2))
print()
print(len(pulpoutput.loc[pulpoutput['Selection Y/N'] == 1]), "out of" ,len(proj_list.index) ,"total projects submitted were shortlisted for execution" )
print()
print(CAPEX_CheckSum.to_string(index = False))

Summary

Total NPV= 268.0

25 out of 30 total projects submitted were shortlisted for execution

 Year  CAPEX Phasing  Constraints  Budget Utilisation%
    1           54.0         55.0                 98.0
    2           77.0         77.0                100.0
    3           44.0         44.0                100.0
    4           21.0         22.0                 95.0


In [16]:
print("Selected Projects")
print("-----------------")
pulpoutput.loc[pulpoutput['Selection Y/N'] == 1]

Selected Projects
-----------------


,Programme,Type,Location,Region,CAPEX,NPV,CAPEX Yr1,CAPEX Yr2,CAPEX Yr3,CAPEX Yr4,Relationship,RelationshipProjID,Selection Y/N,CAPEX Yr1 Selected,CAPEX Yr2 Selected,CAPEX Yr3 Selected,CAPEX Yr4 Selected,NPV Selected
1,Piping,AssetIntegrity,Facility J,Zone4,5,6,4,1,0,0,Contingent,2.0,1.0,4.0,1.0,0.0,0.0,6.0
2,Instrumentation,AssetIntegrity,Facility F,Zone1,10,11,1,5,4,0,Contingent,1.0,1.0,1.0,5.0,4.0,0.0,11.0
4,Debottlenecking,Growth,Facility D,Zone4,5,7,5,0,0,0,0,0.0,1.0,5.0,0.0,0.0,0.0,7.0
5,Piping,AssetIntegrity,Facility C,Zone3,10,12,4,4,2,0,Mutually_Exclusive,3.0,1.0,4.0,4.0,2.0,0.0,12.0
6,Expansion,Growth,Facility C,Zone3,15,20,7,8,0,0,0,0.0,1.0,7.0,8.0,0.0,0.0,20.0
7,Revamp,Growth,Facility F,Zone1,15,16,7,7,1,0,0,0.0,1.0,7.0,7.0,1.0,0.0,16.0
8,Piping,AssetIntegrity,Facility D,Zone4,2,5,1,1,0,0,Mandatory,8.0,1.0,1.0,1.0,0.0,0.0,5.0
9,Revamp,Growth,Facility E,Zone5,3,4,2,1,0,0,Mandatory,9.0,1.0,2.0,1.0,0.0,0.0,4.0
10,Piping,AssetIntegrity,Facility H,Zone3,7,9,3,3,1,0,0,0.0,1.0,3.0,3.0,1.0,0.0,9.0
11,CivilStructural,AssetIntegrity,Facility G,Zone1,4,5,1,1,2,0,0,0.0,1.0,1.0,1.0,2.0,0.0,5.0


In [17]:
print("Dropped Projects")
print("----------------")
pulpoutput.loc[pulpoutput['Selection Y/N'] == 0]

Dropped Projects
----------------


,Programme,Type,Location,Region,CAPEX,NPV,CAPEX Yr1,CAPEX Yr2,CAPEX Yr3,CAPEX Yr4,Relationship,RelationshipProjID,Selection Y/N,CAPEX Yr1 Selected,CAPEX Yr2 Selected,CAPEX Yr3 Selected,CAPEX Yr4 Selected,NPV Selected
3,Mechanical,AssetIntegrity,Facility C,Zone3,10,14,2,4,4,0,Mutually_Exclusive,5.0,0.0,0.0,0.0,0.0,0.0,0.0
15,Instrumentation,AssetIntegrity,Facility H,Zone3,7,6,3,3,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,Instrumentation,AssetIntegrity,Facility J,Zone4,1,2,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,Instrumentation,AssetIntegrity,Facility B,Zone2,2,3,1,0,1,0,Mutually_Exclusive,19.0,0.0,0.0,0.0,0.0,0.0,0.0
28,Debottlenecking,Growth,Facility A,Zone1,20,40,2,3,5,10,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
